# Import libraries

In [1]:
import pandas as pd

# Import dataset

In [2]:
df = pd.read_csv(r"c:\\Users\\LENOVO\\OneDrive\\Desktop\\Tugas Akhir\\Programming\\data\\raw/dataset.csv",
                 low_memory=False)
df.shape

(39219, 36)

# Data Cleaning

### Drop Duplicates

In [3]:
df = df.drop_duplicates('id')
df.shape

(39219, 36)

### Drop tweet that has 0 interaction

In [4]:
df = df[df.reply_to != "[]"]
df.shape

(30867, 36)

### Drop tweet thas from or to @zeniuseducation and @ruangguru

In [5]:
# make reply_to (username and id) columns
df["reply_to_username"] = [
    eval(df.reply_to.tolist()[i])[0]
    ["screen_name"]
    for i in range(0, df.shape[0])]
df["reply_to_id"] = [
    eval(df.reply_to.tolist()[i])[0]["id"]
    for i in range(0, df.shape[0])]
# take only user interaction
df = df[~df.username.str.contains(
    "zeniuseducation|ruangguru")]
df = df[~df.reply_to_username.str.contains(
    "zeniuseducation|ruangguru")]

df.reset_index(inplace=True, drop=True)
df.shape

(8945, 38)

### Tweet Grouping About Ruangguru or Zenius

In [6]:
df_zenius = df[
    (df.tweet.str.contains("zenius") == True)
    &
    ~df.tweet.str.contains(
        "ruangguru|rg|ruang guru")
    ].copy()
df_ruangguru = df[
    df.tweet.str.contains(
        "ruangguru|rg|ruang guru") == True
    &
    ~df.tweet.str.contains("zenius")
    ].copy()

df_zenius.reset_index(inplace=True, drop=True)
df_ruangguru.reset_index(inplace=True, drop=True)

### Transform to Edge List

In [7]:
# function to transform df to edge list form
def transform_edglst(df):
    # source and target column
    edglst = df[
        ['username',
         'reply_to_username']
        ].copy().rename(
            {"username":"Source",
             "reply_to_username":"Target"},
            # to lower case
            axis=1).applymap(
                lambda s:s.lower())
    # weight column
    edglst = edglst.value_counts(
        ).to_frame(
            'Jumlah Interaksi').astype(float)
    return edglst.reset_index()
# apply the function
[edglst_ruangguru,
 edglst_zenius] = [transform_edglst(x)
                   for x in [df_ruangguru,
                             df_zenius]]

### Put the data to CSV format

In [8]:
edglst_ruangguru.to_csv(
    r"c:\Users\LENOVO\OneDrive\Desktop\Tugas Akhir\Programming\data\post-preprocessing\ruangguru-edglst.csv",
    index=False,
    header=True,
)
edglst_zenius.to_csv(
    r"c:\Users\LENOVO\OneDrive\Desktop\Tugas Akhir\Programming\data\post-preprocessing\zenius-edglst.csv",
    index=False,
    header=True,
)